In [3]:
!pip install dask_expr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 5.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:0000:01
  Attempting uninstall: dask
    Found existing installation: dask 2024.2.0
    Uninstalling dask-2024.2.0:
      Successfully uninstalled dask-2024.2.0


In [4]:
!pip install bitstring

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 13.2 MB/s eta 0:00:00


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
import shutil
import h5py
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import hashlib
from concurrent.futures import ProcessPoolExecutor
import os
from scipy.sparse import csr_matrix
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from tqdm.auto import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ogbg-ppi-medium/ppa_graphs
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/RELEASE_v1.txt
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/valid.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/train.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/test.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/mapping/README.md
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/mapping/graphidx2speciesid.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/num-edge-list.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/num-node-list.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/edge-feat.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/edge.csv
/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/graph-label.csv


In [54]:
from bitstring import Bits, BitArray, BitStream, pack
import math
import numpy as np

In [55]:
import numpy as np
import math
from bitstring import BitArray

def Weisfeiler_Leman_with_bit_encoding(adj_mat, num_iter):
    vertex_degrees = np.array(adj_mat.sum(axis=1)).flatten()
    max_degree = max(vertex_degrees)
    bit_length = math.ceil(math.log(max_degree + 1, 2))
    num_vertices = len(vertex_degrees)
    labels = [format(degree, f'0{bit_length}b') for degree in vertex_degrees]

    for _ in range(num_iter):
        new_labels = []
        for v in range(num_vertices):
            neighbors = np.flatnonzero(adj_mat[v])
            neighbor_labels = sorted((labels[n] for n in neighbors), reverse=True)
            new_label = ''.join([labels[v]] + neighbor_labels)
            new_labels.append(new_label)
        labels = new_labels
    
    labels = sorted(labels, reverse=True)
    
    num_iter_encoded = format(num_iter, '08b')
    bit_length_encoded = format(bit_length, '016b')
    labels_encoded = ''.join(labels)
    
    bit_string = num_iter_encoded + bit_length_encoded + labels_encoded
    embedding_string = BitArray(bin=bit_string)
            
    return embedding_string


In [78]:
nlist = [[1, 2, 3], [3, 2, 1]]
str(nlist)


'[[1, 2, 3], [3, 2, 1]]'

In [79]:
def decode_embedding_string(embedding_string):
    num_iter = int(embedding_string[:8].bin, 2)  # extract how many iterations the WL algorithm ran
    bit_length = int(embedding_string[8:24].bin, 2)  # extract the bit length for this graph
    label_bit_string = embedding_string[24:].bin
    num_digits = len(label_bit_string) // bit_length
    digit_list = [int(label_bit_string[x * bit_length: (x + 1) * bit_length], 2) for x in range(num_digits)]
    current_list = [x for x in digit_list]
    embedding = []
    
    for n in range(num_iter):
        new_list = []
        while current_list:
            head = current_list.pop(0)
            head_val = head
            while not isinstance(head_val, int):
                head_val = head_val[0]
            values = current_list[:head_val]
            del current_list[:head_val]
            new_list.append((head, values))
        current_list = new_list
    embedding = current_list
    for n in range(num_iter - 1):
        new_list = [label[0] for label in current_list]
        embedding.extend(new_list)
        current_list = new_list
        
    graph_dict = {}
    for item in embedding:
        if str(item) in graph_dict.keys():
            graph_dict[str(item)] += 1
        else:
            graph_dict[str(item)] = 1
    return graph_dict
        
            

In [80]:
decode_embedding_string(bit_string)

{'(((3, [1, 1, 1]), [(1, [3]), (1, [3]), (1, [3])]), [((1, [3]), [(3, [1, 1, 1])]), ((1, [3]), [(3, [1, 1, 1])]), ((1, [3]), [(3, [1, 1, 1])])])': 1,
 '(((1, [3]), [(3, [1, 1, 1])]), [((3, [1, 1, 1]), [(1, [3]), (1, [3]), (1, [3])])])': 3,
 '((3, [1, 1, 1]), [(1, [3]), (1, [3]), (1, [3])])': 1,
 '((1, [3]), [(3, [1, 1, 1])])': 3,
 '(3, [1, 1, 1])': 1,
 '(1, [3])': 3}

In [24]:
from bitstring import BitArray

def decode_embedding_string(embedding_string):   
    
    num_iter = int(embedding_string[:8].bin, 2)  # extract how many iterations the WL algorithm ran
    bit_length = int(embedding_string[8:24].bin, 2)  # extract the bit length for this graph
    label_bit_string = embedding_string[24:].bin
    
    digit_list = []
    num_digits = len(label_bit_string) // bit_length
    
    for x in range(num_digits):
        digit_list.append(int(label_bit_string[x * bit_length: (x + 1) * bit_length], 2))
    
    graph_dict = {}
    for n in range(num_iter):
        graph_dict[n] = {}
    
    while digit_list:
        key = [digit_list[0]]
        key_string = str(key[0])
        
        for n in range(num_iter):
            if len(digit_list) <= len(key):
                print('Breaking out of loop, not enough digits left')
                break
            key, new_key_string = find_key(digit_list, key_string, key)
            
            if new_key_string not in graph_dict.keys():
                graph_dict[n][new_key_string] = 1
            else:
                graph_dict[n][new_key_string] += 1
            key_string = new_key_string
        digit_list = digit_list[len(key):]
        print(f'Updated digit_list: {digit_list}')
    
    return graph_dict

def find_key(digit_list, key_string, key):
    key_length = len(key)
    val_length = sum(key)
    if key_length + val_length > len(digit_list):
        raise ValueError('Function should stop after all digits are read.')
    new_val = digit_list[key_length: key_length + val_length]
    val_string = "{"
    i = 0
    while i < len(new_val):
        
    key = digit_list[: key_length + val_length]
    new_key_string = '[' + key_string + ':' + ','.join(map(str, new_val)) + ']'
    return key, new_key_string


In [25]:
adj_mat = np.array([
    [0,1,0,0],
    [1,0,1,1],
    [0,1,0,0],
    [0,1,0,0]])
bit_string = Weisfeiler_Leman_with_bit_encoding(adj_mat, 3)
bit_string.bin

'00000011000000000000001011010101011101110111011111010101011111010101011111010101011111010101110101010111011101110111110101011101010101110111011101111101010111010101011101110111'

In [26]:
graph_dict = decode_embedding_string(bit_string)
print(graph_dict)

Updated digit_list: [1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3]
Updated digit_list: [1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3]
Updated digit_list: [1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3]
Updated digit_list: []
{0: {'[3:1,1,1]': 1, '[1:3]': 1}, 1: {'[[3:1,1,1]:1,3,1,3,1,3]': 1, '[[1:3]:3,1,1,1]': 1}, 2: {'[[[3:1,1,1]:1,3,1,3,1,3]:1,3,3,1,1,1,1,3,3,1,1,1,1,3,3,1,1,1]': 1, '[[[1:3]:3,1,1,1]:3,1,1,1,1,3,1,3,1,3]': 1}}


In [ ]:
def chunkify_graphs(graphs, n_iter, parallel):
    if parallel:
        num_cores = os.cpu_count()
        num_graphs = len(graphs)
        steps = []
        steps.append(0)
        step = 0
        for i in range(num_cores):
            step += num_graphs//num_cores
            steps.append(step)
        steps[-1] = num_graphs
        for i in range(num_cores):
            yield (graphs[steps[i]:steps[i+1]], n_iter)
    else:
        yield (graphs, n_iter)


def process_vertex(graph, vertex, vertex_labels):
    neighbors = graph[vertex].indices
    neighbor_labels = [vertex_labels[n] for n in neighbors]
    sorted_neighbor_labels = sorted_neighbor_labels
    concatenated_labels = ''.join(sorted(neighbor_labels))
    return stable_hash(concatenated_labels)
def process_graph(graph, n_iter):
    vertex_degrees = np.array(graph.sum(axis=1)).flatten()
    vertex_labels = [degree for degree in vertex_degrees]
    num_vertices = len(vertex_labels)
    graph_hash_values = np.empty(shape=(n_iter, num_vertices), dtype='<U32')
    for i in range(n_iter):
        new_vertex_labels  = []
        for vertex in range(num_vertices):
            new_vertex_labels.append(process_vertex(graph, vertex, vertex_labels))
        vertex_labels = np.array([vertex_label for vertex_label in new_vertex_labels])
        graph_hash_values[i, :] = vertex_labels
    hashes, counts = np.unique(graph_hash_values, return_counts = True)
    graph_embedding_dict = dict(zip(hashes, counts))
    return graph_embedding_dict
def process_chunk_of_graphs(chunk_of_graphs_and_n_iter):
    chunk_of_graphs, n_iter = chunk_of_graphs_and_n_iter
    """Process a chunk of graphs and return their hash counts"""
    results = []
    for graph in chunk_of_graphs:
        results.append(process_graph(graph, n_iter))
    return results
def graph_dataset_WL_embedding(graphs, n_iter, parallel = False):
    num_cores = os.cpu_count()
    for graph in graphs:
        if not isinstance(graph, csr_matrix):
            raise TypeError("All graphs need to be csr_matrix")

    
    # Calculate chunk size
    chunks_of_graphs_and_n_iter = list(chunkify_graphs(graphs, n_iter, parallel))
    chunk_results = []
    if parallel:
        with ProcessPoolExecutor() as executor:
            # Map each chunk to a process
            chunk_results = list(executor.map(process_chunk_of_graphs, chunks_of_graphs_and_n_iter))
    else:
        for chunk in chunks_of_graphs_and_n_iter:
            chunk_results.extend(process_chunk_of_graphs(chunk))
  
    graph_embeddings = [embedding for chunk in chunk_results for embedding in chunk]
    return graph_embeddings       
 

In [ ]:
#graphidx2species = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/mapping/graphidx2speciesid.csv')
#valid = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/valid.csv', header = None).astype(np.uint32)
#test = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/test.csv', header = None).astype(np.uint32)
#train = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/valid.csv', header = None).astype(np.uint32)
graph_label= pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/graph-label.csv', header = None).astype(np.uint8)

num_node_list = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/num-node-list.csv', header = None).astype(np.uint16)
num_edge_list = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/num-edge-list.csv', header = None).astype(np.uint64)
edge = pd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/edge.csv', header=None).astype(np.uint16)
edge_feat = dd.read_csv('/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/raw/edge-feat.csv', header = None).astype(np.float32)

In [ ]:
gc.collect()

In [ ]:
num_node_list = num_node_list.to_numpy().flatten()
num_edge_list = num_edge_list.to_numpy().flatten()
graph_label = graph_label.to_numpy().flatten()


In [ ]:
len(graph_label)

In [ ]:
edge_feat = edge_feat.compute()

In [ ]:
gc.collect()

In [ ]:
gc_interval = 50
with h5py.File('graph_data.hdf5', 'w') as f:
    num_edge_accum = 0  # Initialize accumulator for edge indices
    for i, (num_node, num_edge) in enumerate(tqdm(zip(num_node_list, num_edge_list), total=len(num_node_list))):
        # Create a group for each graph
        grp = f.create_group(f'graph_{i}')
        
        # Generate the CSR matrix
        start, stop = int(num_edge_accum), int(num_edge_accum + num_edge)
        rows = edge.iloc[start:stop, 0].values
        cols = edge.iloc[start:stop, 1].values
        data = np.ones(len(rows), dtype=bool)
        csr_graph = csr_matrix((data, (rows, cols)), shape=(num_node, num_node))
        
        embedding = process_graph(csr_graph, 3)
        embedding_list_bytes = [(hash.encode('utf-8'), count) for hash, count in list(embedding.items())]
        dt = np.dtype([('hash', 'S32'), ('count', np.int32)])  # 'S32' for 32-byte strings
        embedding_array = np.array(embedding_list_bytes, dtype=dt)
        grp.create_dataset('embedding', data = embedding_array)
        
        # Store CSR matrix components in the group
        grp.create_dataset('data', data=csr_graph.data)
        grp.create_dataset('indices', data=csr_graph.indices)
        grp.create_dataset('indptr', data=csr_graph.indptr)
        grp.create_dataset('shape', data=csr_graph.shape)
        
        # Store edge features
        edge_feats = edge_feat.iloc[start:stop, :].values
        grp.create_dataset('edge_feat', data=edge_feats)
        
        # Optionally, store other graph attributes
        grp.attrs['num_nodes'] = num_node
        grp.attrs['num_edges'] = num_edge
        grp.attrs['graph_label'] = graph_label[i]
        
        # Update accumulators
        num_edge_accum += num_edge

        # Optional: Clear memory, if necessary
        if (i + 1) % gc_interval == 0:
            gc.collect()

In [ ]:
import shutil
import os

# Define source paths
src_graphidx2species = '/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/mapping/graphidx2speciesid.csv'
src_valid = '/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/valid.csv'
src_test = '/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/test.csv'
src_train = '/kaggle/input/ogbg-ppi-medium/ogbg_ppi_medium/ogbg_ppi_medium/split/species/train.csv'  # Assuming this path is correct and meant to be 'valid.csv'

# Define destination directory
dest_dir = '/kaggle/working/'

# Ensure the destination directory exists
os.makedirs(dest_dir, exist_ok=True)

# Copy files
shutil.copy(src_graphidx2species, os.path.join(dest_dir, 'graphidx2speciesid.csv'))
shutil.copy(src_valid, os.path.join(dest_dir, 'valid.csv'))
shutil.copy(src_test, os.path.join(dest_dir, 'test.csv'))
shutil.copy(src_train, os.path.join(dest_dir, 'train.csv'))  # Note: This copies 'valid.csv' as 'train.csv' based on your paths. Adjust if needed.
